# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up Turi Create

In [1]:
import turicreate as tc

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = tc.SFrame('data/home_data.sframe/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float and then to int, before using it below
sales['floors'] = sales['floors'].astype(float).astype(int)

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]

    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return feature_matrix, output_array

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return predictions

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.], [4.,12.,15.]])
print(X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print(norms)

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [8]:
print(X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [138]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    return feature_matrix / norms, norms

To test the function, run the following:

In [139]:
features, norms = normalize_features(np.array([[3.,6.,9.], [4.,8.,12.]]))
print(features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [16]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [17]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [18]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [20]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [25]:
ros = []

for i in range(simple_feature_matrix.shape[1]):
    feature_i = simple_feature_matrix[:, i]
    roi = (feature_i * (output - prediction + weights[i] * feature_i)).sum()
    ros.append(roi)

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [28]:
# l1 >= 2|ro2| and l1 <= 2 ro1
print(ros[1] * 2, ros[2] * 2)

175878941.54598215 161933397.3519313


In [26]:
ros

[79400300.03492916, 87939470.77299108, 80966698.67596565]

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [29]:
max(2 * abs(ros[1]), 2 * abs(ros[2]))

175878941.54598215

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [30]:
predict_output?

In [31]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = feature_matrix[:, i]
    weight_i = weights[i]
    ro_i = (feature_i * (output - prediction + weight_i * feature_i)).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [32]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                    np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4755588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [71]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights
    
    while True:
        max_weight_change = 0.0
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            
            weight_change = np.abs(old_weights_i - weights[i])
            if max_weight_change < weight_change:
                max_weight_change = weight_change
        
        print("Max Weight Change: {:.2f}".format(max_weight_change))
        if max_weight_change < tolerance:
            break
    
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [129]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [133]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [134]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

Max Weight Change: 8187106.54
Max Weight Change: 1313989.49
Max Weight Change: 1142308.44
Max Weight Change: 990623.28
Max Weight Change: 856748.66
Max Weight Change: 738728.56
Max Weight Change: 634812.92
Max Weight Change: 543436.56
Max Weight Change: 463200.21
Max Weight Change: 392853.49
Max Weight Change: 331279.55
Max Weight Change: 299065.62
Max Weight Change: 279293.86
Max Weight Change: 260686.75
Max Weight Change: 243192.30
Max Weight Change: 226758.64
Max Weight Change: 211334.47
Max Weight Change: 196869.27
Max Weight Change: 183313.66
Max Weight Change: 170619.48
Max Weight Change: 158740.06
Max Weight Change: 147630.24
Max Weight Change: 137246.54
Max Weight Change: 129094.23
Max Weight Change: 126911.71
Max Weight Change: 124117.54
Max Weight Change: 120835.83
Max Weight Change: 117173.64
Max Weight Change: 113223.00
Max Weight Change: 109062.73
Max Weight Change: 104760.05
Max Weight Change: 100371.98
Max Weight Change: 95946.66
Max Weight Change: 91524.42
Max Weight Ch

In [137]:
weights

array([ 1.16830801e+07,  1.04939636e+08, -2.93366540e+07])

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [78]:
pred = predict_output(normalized_simple_feature_matrix, weights)

In [79]:
residuals = output - pred
rss = (residuals * residuals).sum()

In [80]:
print(rss)

1436640985853594.5


In [126]:
1436640985853594.5 / 1e15

1.4366409858535945

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [81]:
train_data, test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [105]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [111]:
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, 'price')
(normalized_all_feature_matrix, all_norms) = normalize_features(all_feature_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [84]:
initial_weights = np.zeros(len(all_features) + 1)

In [85]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                               initial_weights, l1_penalty=1e7, tolerance=1)

Max Weight Change: 71114625.75
Max Weight Change: 7209840.30
Max Weight Change: 5580201.02
Max Weight Change: 4362686.81
Max Weight Change: 3427793.80
Max Weight Change: 2710319.91
Max Weight Change: 2160361.40
Max Weight Change: 1739480.11
Max Weight Change: 1604145.49
Max Weight Change: 1504184.14
Max Weight Change: 1424403.62
Max Weight Change: 1360120.95
Max Weight Change: 1307729.51
Max Weight Change: 1264452.45
Max Weight Change: 1228152.53
Max Weight Change: 1197185.59
Max Weight Change: 1170287.53
Max Weight Change: 1146487.14
Max Weight Change: 1125038.95
Max Weight Change: 1105371.38
Max Weight Change: 1087046.84
Max Weight Change: 1069730.88
Max Weight Change: 1053168.51
Max Weight Change: 1037165.86
Max Weight Change: 1021576.09
Max Weight Change: 1006288.50
Max Weight Change: 991220.17
Max Weight Change: 976309.51
Max Weight Change: 961511.26
Max Weight Change: 946792.67
Max Weight Change: 932130.54
Max Weight Change: 917508.99
Max Weight Change: 902917.64
Max Weight Chang

Max Weight Change: 82530.91
Max Weight Change: 82514.68
Max Weight Change: 82498.45
Max Weight Change: 82482.23
Max Weight Change: 82466.01
Max Weight Change: 82449.79
Max Weight Change: 82433.58
Max Weight Change: 82417.37
Max Weight Change: 82401.16
Max Weight Change: 82384.96
Max Weight Change: 82368.76
Max Weight Change: 82352.56
Max Weight Change: 82336.36
Max Weight Change: 82320.17
Max Weight Change: 82303.98
Max Weight Change: 82287.80
Max Weight Change: 82271.61
Max Weight Change: 82255.44
Max Weight Change: 82239.26
Max Weight Change: 82223.09
Max Weight Change: 82206.92
Max Weight Change: 82190.75
Max Weight Change: 82174.59
Max Weight Change: 82158.43
Max Weight Change: 82142.27
Max Weight Change: 82126.12
Max Weight Change: 82109.97
Max Weight Change: 82093.82
Max Weight Change: 82077.68
Max Weight Change: 82061.54
Max Weight Change: 82045.40
Max Weight Change: 82029.26
Max Weight Change: 82013.13
Max Weight Change: 81997.01
Max Weight Change: 81980.88
Max Weight Change: 8

Max Weight Change: 77756.43
Max Weight Change: 77741.14
Max Weight Change: 77725.85
Max Weight Change: 77710.57
Max Weight Change: 77695.29
Max Weight Change: 77680.01
Max Weight Change: 77664.73
Max Weight Change: 77649.46
Max Weight Change: 77634.19
Max Weight Change: 77618.92
Max Weight Change: 77603.66
Max Weight Change: 77588.40
Max Weight Change: 77573.14
Max Weight Change: 77557.88
Max Weight Change: 77542.63
Max Weight Change: 77527.38
Max Weight Change: 77512.14
Max Weight Change: 77496.89
Max Weight Change: 77481.65
Max Weight Change: 77466.42
Max Weight Change: 77451.18
Max Weight Change: 77435.95
Max Weight Change: 77420.72
Max Weight Change: 77405.50
Max Weight Change: 77390.28
Max Weight Change: 77375.06
Max Weight Change: 77359.84
Max Weight Change: 77344.63
Max Weight Change: 77329.42
Max Weight Change: 77314.21
Max Weight Change: 77299.01
Max Weight Change: 77283.81
Max Weight Change: 77268.61
Max Weight Change: 77253.41
Max Weight Change: 77238.22
Max Weight Change: 7

Max Weight Change: 73157.37
Max Weight Change: 73142.99
Max Weight Change: 73128.60
Max Weight Change: 73114.22
Max Weight Change: 73099.85
Max Weight Change: 73085.47
Max Weight Change: 73071.10
Max Weight Change: 73056.73
Max Weight Change: 73042.36
Max Weight Change: 73028.00
Max Weight Change: 73013.64
Max Weight Change: 72999.28
Max Weight Change: 72984.92
Max Weight Change: 72970.57
Max Weight Change: 72956.22
Max Weight Change: 72941.87
Max Weight Change: 72927.53
Max Weight Change: 72913.19
Max Weight Change: 72898.85
Max Weight Change: 72884.51
Max Weight Change: 72870.18
Max Weight Change: 72855.85
Max Weight Change: 72841.52
Max Weight Change: 72827.20
Max Weight Change: 72812.88
Max Weight Change: 72798.56
Max Weight Change: 72784.24
Max Weight Change: 72769.93
Max Weight Change: 72755.62
Max Weight Change: 72741.31
Max Weight Change: 72727.01
Max Weight Change: 72712.70
Max Weight Change: 72698.40
Max Weight Change: 72684.11
Max Weight Change: 72669.81
Max Weight Change: 7

Max Weight Change: 68749.16
Max Weight Change: 68735.64
Max Weight Change: 68722.13
Max Weight Change: 68708.61
Max Weight Change: 68695.10
Max Weight Change: 68681.59
Max Weight Change: 68668.09
Max Weight Change: 68654.58
Max Weight Change: 68641.08
Max Weight Change: 68627.58
Max Weight Change: 68614.09
Max Weight Change: 68600.59
Max Weight Change: 68587.10
Max Weight Change: 68573.62
Max Weight Change: 68560.13
Max Weight Change: 68546.65
Max Weight Change: 68533.17
Max Weight Change: 68519.69
Max Weight Change: 68506.22
Max Weight Change: 68492.74
Max Weight Change: 68479.27
Max Weight Change: 68465.81
Max Weight Change: 68452.34
Max Weight Change: 68438.88
Max Weight Change: 68425.42
Max Weight Change: 68411.97
Max Weight Change: 68398.51
Max Weight Change: 68385.06
Max Weight Change: 68371.62
Max Weight Change: 68358.17
Max Weight Change: 68344.73
Max Weight Change: 68331.29
Max Weight Change: 68317.85
Max Weight Change: 68304.41
Max Weight Change: 68290.98
Max Weight Change: 6

...

Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.34
Max Weight Change: 1.33
Max Weight Change: 1.33
Max Weight Change: 1.33
Max Weight Change: 1.33
Max Weight Change: 1.33
Max Weight Change: 1.33
Max Weight Change: 1.33
Max Weight Change: 1.33
Max Weight Change: 1.33
Max Weight Change: 1.33
Max Weight Chang

Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.25
Max Weight Change: 1.24
Max Weight Change: 1.24
Max Weight Change: 1.24
Max Weight Change: 1.24
Max Weight Change: 1.24
Max Weight Change: 1.24
Max Weight Change: 1.24
Max Weight Chang

Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.16
Max Weight Change: 1.15
Max Weight Change: 1.15
Max Weight Change: 1.15
Max Weight Change: 1.15
Max Weight Change: 1.15
Max Weight Chang

Max Weight Change: 1.09
Max Weight Change: 1.09
Max Weight Change: 1.09
Max Weight Change: 1.09
Max Weight Change: 1.09
Max Weight Change: 1.09
Max Weight Change: 1.09
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Change: 1.08
Max Weight Chang

Max Weight Change: 1.01
Max Weight Change: 1.01
Max Weight Change: 1.01
Max Weight Change: 1.01
Max Weight Change: 1.01
Max Weight Change: 1.01
Max Weight Change: 1.01
Max Weight Change: 1.01
Max Weight Change: 1.01
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00
Max Weight Change: 1.00


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [92]:
all_features_with0 = np.array(['0', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
                               'floors', 'waterfront', 'view', 'condition', 'grade',
                               'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated'])

In [94]:
all_features_with0[weights1e7 != 0.0]

array(['0', 'sqft_living', 'waterfront', 'view', 'yr_built'], dtype='<U13')

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [95]:
initial_weights = np.zeros(len(all_features) + 1)

In [96]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                               initial_weights, l1_penalty=1e8, tolerance=1)

Max Weight Change: 71114625.75
Max Weight Change: 0.00


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [97]:
all_features_with0[weights1e8 != 0.0]

array(['0'], dtype='<U13')

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [113]:
initial_weights = np.zeros(len(all_features) + 1)

In [114]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                               initial_weights, l1_penalty=1e4, tolerance=5e5)

Max Weight Change: 71114625.75
Max Weight Change: 8473404.79
Max Weight Change: 6827708.95
Max Weight Change: 5579304.36
Max Weight Change: 4568844.39
Max Weight Change: 3740917.36
Max Weight Change: 3061920.02
Max Weight Change: 2506070.95
Max Weight Change: 2052268.32
Max Weight Change: 1682966.09
Max Weight Change: 1383523.15
Max Weight Change: 1295374.28
Max Weight Change: 1256078.78
Max Weight Change: 1227003.45
Max Weight Change: 1205588.72
Max Weight Change: 1189802.65
Max Weight Change: 1178029.79
Max Weight Change: 1169045.34
Max Weight Change: 1161866.42
Max Weight Change: 1155751.03
Max Weight Change: 1150140.53
Max Weight Change: 1144622.13
Max Weight Change: 1138898.71
Max Weight Change: 1132764.22
Max Weight Change: 1126083.63
Max Weight Change: 1118776.83
Max Weight Change: 1110805.58
Max Weight Change: 1102163.08
Max Weight Change: 1092865.65
Max Weight Change: 1082946.17
Max Weight Change: 1072448.85
Max Weight Change: 1061425.25
Max Weight Change: 1049931.11
Max Weigh

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [115]:
all_features_with0[weights1e4 != 0.0]

array(['0', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated'], dtype='<U13')

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [116]:
weights1e4_normed = weights1e4 / all_norms
weights1e7_normed = weights1e7 / all_norms
weights1e8_normed = weights1e8 / all_norms

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [124]:
print(weights1e7_normed[3])

281.48947072233756


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [106]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [118]:
def rss(pred, actual):
    return ((pred - actual) * (pred - actual)).sum()

In [119]:
pred_1e4 = predict_output(test_feature_matrix, weights1e4_normed)
print(rss(pred_1e4, test_output))

227627270917284.5


In [120]:
pred_1e7 = predict_output(test_feature_matrix, weights1e7_normed)
print(rss(pred_1e7, test_output))

232919761937401.16


In [121]:
pred_1e8 = predict_output(test_feature_matrix, weights1e8_normed)
print(rss(pred_1e8, test_output))

537166150034084.9


***QUIZ QUESTION***

Which model performed best on the test data?